Dữ liệu
https://drive.google.com/drive/folders/1vA9yta8dDBP8eJOfwh8JItJuqwSCyMfS?usp=sharing

#Xử lý dữ liệu

In [ ]:
!unzip "/content/drive/My Drive/D/Đồ Án.zip" -d "./Data"
%cp "/content/Data/Đồ Án/NLP 02/Stats.txt" -d '/content/Data'
!pip install unrar
!unrar x "/content/Data/Đồ Án/NLP 02/Test.rar" -d "./Data"
!unrar x "/content/Data/Đồ Án/NLP 02/Train.rar" -d "./Data"

In [ ]:
!git clone https://github.com/stopwords/vietnamese-stopwords.git
%cp "/content/vietnamese-stopwords/vietnamese-stopwords-dash.txt" -d "/content/Data"

In [ ]:
def PhanLoai(Topic):
  TopicIDs = {"Am nhac":"AN", "Am thuc":"AT", "Bat dong san":"BDS", "Bong da":"BD", "Chung khoan":"CK",
              "Cum ga":"CG", "Cuoc song do day":"CSDD", "Du hoc":"DH", "Du lich":"DL", "Duong vao WTO":"DVW",
              "Gia dinh":"GD", "Giai tri tin hoc":"GTTH", "Giao duc":"GDu", "Gioi tinh":"GT", "Hackers va Virus":"HV",
              "Hinh su":"HS", "Khong gian song":"KGS", "Kinh doanh quoc te":"KDQT", "Lam dep":"LD", "Loi song":"LS",
              "Mua sam":"MS", "My thuat":"MT", "San khau dien anh":"SKDA", "San pham tin hoc moi":"SPTHM", "Tennis":"T",
              "The gioi tre":"TGT", "Thoi trang":"TT"}
  return TopicIDs[Topic]

In [ ]:
# Hàm tiền xử lý dữ liệu
def Preprocessing(path):
  files = []
  ids = []

  stopword = []
  with open("/content/Data/vietnamese-stopwords-dash.txt", "r") as f:
    stopword = f.read().splitlines()

# Lấy các file .txt
  # r=root, d=directories, f = files
  for r, d, f in os.walk(path):
      for folder in d:
          id = PhanLoai(folder)
          for r1, d1, f in os.walk(os.path.join(r, folder)):
            for file in f:
              files.append(os.path.join(r1, file))
              ids.append(id)

  X = []
  for file in tqdm(files, desc='Processing', position=0):
    with open(file, 'r', encoding='utf-16') as f:
      # Ghép các từ riêng lẻ thành cụm từ có nghĩa
      txt = ViTokenizer.tokenize(f.read())
      # Xóa các ký tự đặc biệt không phải chữ hoặc số
      txt = strip_non_alphanum(txt).lower().strip()
      # Xóa ký tự là số
      txt = strip_numeric(txt)
      # Xóa từ có số và chữ, vd a1 a2
      txt = split_alphanum(txt)
      # Xóa các chữ đứng riêng lẻ
      txt = strip_short(txt, minsize=2)
      # Chuẩn hóa để mỗi từ cách nhau một khoảng trắng
      txt = strip_multiple_whitespaces(txt)
      # Tách cầu thành từ riêng lẻ và xóa stopword
      txt = txt.split()
      txt = [word for word in txt if word not in stopword]
      # Hợp lại thành câu
      txt = " ".join(txt)
      X.append(txt)

  return X, ids

In [ ]:
# Xử lý dữ liệu
path = "/content/Data/new train"
X_train, Y_train = Preprocessing(path)
path = "/content/Data/new test"
X_test, Y_test = Preprocessing(path)

In [ ]:
# Lưu lại dữ liệu đã tiền xử lý
pickle.dump(X_train, open('Data/X_train.pkl', 'wb'))
pickle.dump(Y_train, open('Data/Y_train.pkl', 'wb'))
pickle.dump(X_test, open('Data/X_test.pkl', 'wb'))
pickle.dump(Y_test, open('Data/Y_test.pkl', 'wb'))

#Load data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cp "/content/drive/MyDrive/Data_Do_An_ML/X_train.pkl" -d "/content/Data"
%cp "/content/drive/MyDrive/Data_Do_An_ML/X_test.pkl" -d "/content/Data"
%cp "/content/drive/MyDrive/Data_Do_An_ML/Y_train.pkl" -d "/content/Data"
%cp "/content/drive/MyDrive/Data_Do_An_ML/Y_test.pkl" -d "/content/Data"
%cp "/content/drive/MyDrive/Data_Do_An_ML/X_train_trans.pkl" -d "/content/Data"
%cp "/content/drive/MyDrive/Data_Do_An_ML/X_test_trans.pkl" -d "/content/Data"
%cp "/content/drive/MyDrive/Data_Do_An_ML/X_train_tf.pkl" -d "/content/Data"
%cp "/content/drive/MyDrive//Data_Do_An_ML/X_test_tf.pkl" -d "/content/Data"

#K Nearest Neighbors

In [8]:
import time
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, plot_confusion_matrix

In [6]:
X_train = pickle.load(open('/content/Data/X_train_trans.pkl', 'rb'))
X_test = pickle.load(open('/content/Data/X_test_trans.pkl', 'rb'))
Y_train = pickle.load(open('/content/Data/Y_train.pkl', 'rb'))
Y_test =  pickle.load(open('/content/Data/Y_test.pkl', 'rb'))

In [ ]:
start = time.time()
a = [1, 3, 5, 7, 9, 11, 13, 15 ,17 ,19]
b = ["euclidean", "cosine", "minkowski"]
c = ["uniform", "distance"]
df = pd.DataFrame(columns=('time', 'n_neibors', 'metric', 'weights', 'f1_score', 'accuracy_score', 'precision_score', 'recall_score'))
for i in tqdm(a, position=0, leave=True):
  for j in b:
    for l in c:
      t = time.time()
      model = KNeighborsClassifier(n_neighbors= i, metric= j, weights= l)
      model.fit(X_train, Y_train)
      Y_pred = model.predict(X_test)
      for k in ["micro", "macro", "weighted"]:
        d1 = f1_score(Y_test, Y_pred, average=k)
        d2 = accuracy_score(Y_test, Y_pred)
        d3 = precision_score(Y_test, Y_pred, average = k)
        d4 = recall_score(Y_test, Y_pred, average = k)
        row = pd.Series({'time':time.time()-t, 'n_neibors':i, 'metric':j, 'average':k, 'weights':l, 'f1_score':d1, 'accuracy_score':d2, 'precision_score':d3, 'recall_score':d4})
        df = df.append(row, ignore_index=True)
print(time.time() - start)

In [ ]:
start = time.time()
a = [1, 3, 5, 7, 9, 11, 13, 15 ,17 ,19]
b = ["euclidean", "cosine", "minkowski"]
c = ["uniform", "distance"]
c1=c2= c3= c4 = 0
df = pd.DataFrame(columns=('time', 'n_neibors', 'metric', 'weights', 'f1_score', 'accuracy_score', 'precision_score', 'recall_score'))
for i in a:
  for j in b:
    for l in c:
      t = time.time()
      model = KNeighborsClassifier(n_neighbors= i, metric= j, weights= l)
      model.fit(X_train, Y_train)
      Y_pred = model.predict(X_test)
      for k in ["micro", "macro", "weighted"]:
        d1 = f1_score(Y_test, Y_pred, average=k)
        d2 = accuracy_score(Y_test, Y_pred)
        d3 = precision_score(Y_test, Y_pred, average = k)
        d4 = recall_score(Y_test, Y_pred, average = k)
        if (d1>c1):
          m1 = model
          c1 = d1
        if (d2>c2):
          m2 = model
          c2 = d2
        if (d3>c3):
          m3 = model
          c3 = d3
        if (d4>c4):
          m4 = model
          c3 = d4
        row = pd.Series({'time':time.time()-t, 'n_neibors':i, 'metric':j, 'average':k, 'weights':l, 'f1_score':d1, 'accuracy_score':d2, 'precision_score':d3, 'recall_score':d4})
        df = df.append(row, ignore_index=True)
        print("done")
print(time.time() - start)

In [ ]:
print(df)

In [ ]:
df.to_csv('/content/Data/KNN/export.csv', index=False)
pickle.dump(m1, open('/content/Data/KNN/model_KNN_max_f1.pkl', 'wb'))
pickle.dump(m2, open('/content/Data/KNN/model_KNN_max_acc.pkl', 'wb'))
pickle.dump(m3, open('/content/Data/KNN/model_KNN_max_pre.pkl', 'wb'))
pickle.dump(m4, open('/content/Data/KNN/model_KNN_max_rec.pkl', 'wb'))

In [ ]:
title = "CM"
fig, ax = plt.subplots(figsize=(14,14))
disp = plot_confusion_matrix(model, X_test, Y_test,cmap=plt.cm.Blues,ax=ax)
disp.ax_.set_title(title)
print(title)
print(disp.confusion_matrix)

#Logistic Regression

In [1]:
import pickle
import pandas
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, accuracy_score

In [2]:
X_train = pickle.load(open('/content/Data/X_train.pkl', 'rb'))
X_test = pickle.load(open('/content/Data/X_test.pkl', 'rb'))
Y_test = pickle.load(open('/content/Data/Y_test.pkl', 'rb'))
Y_train = pickle.load(open('/content/Data/Y_train.pkl', 'rb'))
X_train_trans = pickle.load(open('/content/Data/X_train_tf.pkl', 'rb'))
X_test_trans = pickle.load(open('/content/Data/X_test_tf.pkl', 'rb'))

In [ ]:
b = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
for k in tqdm(b, position=0, leave=True):
  model = LogisticRegression(solver=k)
  model.fit(X_train_trans, Y_train)
  f1 = f1_score(Y_test, model.predict(X_test_trans), average="micro")
  print(k, f1)
  #title = "CM"
  #fig, ax = plt.subplots(figsize=(14,14))
  #plot_confusion_matrix(model, X_test, Y_test,cmap=plt.cm.Blues,ax=ax)
  #plt.show()

In [ ]:
b = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
for k in tqdm(b, position=0, leave=True):
  model = LogisticRegression(solver=k)
  model.fit(X_train_trans, Y_train)
  precision = precision_score(Y_test, model.predict(X_test_trans), average="micro")
  print(k, precision)
  #title = "CM"
  #fig, ax = plt.subplots(figsize=(14,14))
  #plot_confusion_matrix(model, X_test, Y_test,cmap=plt.cm.Blues,ax=ax)
  #plt.show()

In [ ]:
b = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
for k in tqdm(b, position=0, leave=True):
  model = LogisticRegression(solver=k)
  model.fit(X_train_trans, Y_train)
  recall = recall_score(Y_test, model.predict(X_test_trans), average="micro")
  print(k, recall)
  #title = "CM"
  #fig, ax = plt.subplots(figsize=(14,14))
  #plot_confusion_matrix(model, X_test, Y_test,cmap=plt.cm.Blues,ax=ax)
  #plt.show()

In [ ]:
b = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
for k in tqdm(b, position=0, leave=True):
  model = LogisticRegression(solver=k)
  model.fit(X_train_trans, Y_train)
  accuracy = accuracy_score(Y_test, model.predict(X_test_trans), normalize="True")
  print(k, accuracy)
  #title = "CM"
  #fig, ax = plt.subplots(figsize=(14,14))
  #plot_confusion_matrix(model, X_test, Y_test,cmap=plt.cm.Blues,ax=ax)
  #plt.show()

#Support Vector Machine

In [9]:
# import những thư viện cần thiết
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.metrics import plot_confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score, classification_report

In [10]:
# đọc dữ liệu đã biến đổi sang dạng vector và các dữ liệu liên quan
X_test = pickle.load(open('/content/Data/X_test.pkl', 'rb'))
X_train = pickle.load(open('/content/Data/X_train.pkl', 'rb'))
Y_train = pickle.load(open('/content/Data/Y_train.pkl', 'rb'))
Y_test = pickle.load(open('/content/Data/Y_test.pkl', 'rb'))

In [ ]:
for kernel in tqdm(['sigmoid', 'poly', 'rbf','linear']):
  SVM = Pipeline([('vect', CountVectorizer(ngram_range=(1,1),max_df=0.8, max_features=None)),('tfidf', TfidfTransformer()),('clf', SVC(kernel=kernel, verbose=True,gamma='scale'))])
  SVM.fit(X_train,Y_train)
  # Lưu lại model
  pickle.dump(SVM, open('/content/Data/SVM/modelSVM_{}_Pipeline.pkl'.format(kernel), 'wb'))

In [ ]:
SVM= ['sigmoid', 'poly', 'rbf','linear']
df=pd.DataFrame(columns=(' ','','Precision', 'F1 score', 'Recall', 'Accuracy', 'Accuracy_false'))
for svm in SVM :
  print('SVM: ',svm)
  model = pickle.load(open('/content/Data/SVM/modelSVM_{}_Pipeline.pkl'.format(svm), 'rb'))
  Y_pred = model.predict(X_test)
  row = pd.Series({' ':svm})
  df = df.append(row, ignore_index = True )
  for a in ('macro','micro','weighted'):
    print("Tham số truyền vào average: ", a)
    p = precision_score(Y_test, Y_pred,average= a)
    f = f1_score(Y_test, Y_pred, average= a)
    r = recall_score(Y_test, Y_pred, average= a)
    row = pd.Series({'':a,'Precision':p, 'F1 score':f, 'Recall':r})
    df = df.append(row, ignore_index = True )
    print("Precision: ",p)
    print("F1 score: ", f)
    print("Recall: ", r)
  cm = confusion_matrix(Y_test, model.predict(X_test))
  fig, ax = plt.subplots(figsize=(14,14))
  plot_confusion_matrix(model, X_test, Y_test,cmap=plt.cm.Blues,ax=ax)
  plt.show()
  # Tạo một dataframe dùng để lưu lại kết quả confusion_matrix
  da = pd.DataFrame(cm)
  # lưu lại kết quả confusion_matrix tập train
  da.to_csv("/content/Data/SVM/SVM_Pipeline_{}_test_cm.csv".format(svm))
  ac = accuracy_score(Y_test,Y_pred, normalize=True)
  acf = accuracy_score(Y_test,Y_pred, normalize=False)
  row = pd.Series({'Accuracy':ac, 'Accuracy_false':acf})
  df = df.append(row, ignore_index = True )
  print("Accuracy: ", ac)
  print("Accuracy_false: ", acf)
df.to_csv('/content/Data/SVM/SVM_Kq_Pipeline.csv', index = False)

#Naive Bayes

In [ ]:
# import những thư viện cần thiết
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.metrics import plot_confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB, BernoulliNB, CategoricalNB
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score, classification_report

In [ ]:
# đọc dữ liệu đã biến đổi sang dạng vector và các dữ liệu liên quan
X_test = pickle.load(open('/content/Data/X_test.pkl', 'rb'))
X_train = pickle.load(open('/content/Data/X_train.pkl', 'rb'))
Y_train = pickle.load(open('/content/Data/Y_train.pkl', 'rb'))
Y_test = pickle.load(open('/content/Data/Y_test.pkl', 'rb'))

In [ ]:
# dùng pipeline để training mô hình NB GaussianNB
NB = Pipeline([('vect', CountVectorizer(ngram_range=(1,1),max_df=0.8, max_features=None)),('tfidf', TfidfTransformer()),('clf',MultinomialNB())])
NB = NB.fit(X_train, Y_train)
pickle.dump(NB, open('/content/Data/NB/MultiNB_Pipeline.pkl', 'wb'))

In [ ]:
# dùng pipeline để training mô hình NB ComplementNB
NB = Pipeline([('vect', CountVectorizer(ngram_range=(1,1),max_df=0.8, max_features=None)),('tfidf', TfidfTransformer()),('clf',ComplementNB())])
NB = NB.fit(X_train, Y_train)
pickle.dump(NB, open('/content/Data/NB/ComplementNB_Pipeline.pkl', 'wb'))

In [ ]:
# dùng pipeline để training mô hình NB BernoulliNB
NB = Pipeline([('vect', CountVectorizer(ngram_range=(1,1),max_df=0.8, max_features=None)),('tfidf', TfidfTransformer()),('clf',BernoulliNB())])
NB = NB.fit(X_train, Y_train)
pickle.dump(NB, open('/content/Data/NB/BernoulliNB_Pipeline.pkl', 'wb'))

In [ ]:
N = ['MultinomialNB', 'ComplementNB', 'BernoulliNB']
df=pd.DataFrame(columns=(' ','','Precision', 'F1 score', 'Recall', 'Accuracy', 'Accuracy_false'))
for NB in N :
  print('Naive bayes: ',NB)
  model = pickle.load(open('/content/Data/NB/{}_Pipeline.pkl'.format(NB), 'rb'))
  Y_pred = model.predict(X_test)
  row = pd.Series({' ':NB})
  df = df.append(row, ignore_index = True )
  for a in ('macro','micro','weighted'):
    print("Tham số truyền vào average: ", a)
    p = precision_score(Y_test, Y_pred,average= a)
    f = f1_score(Y_test, Y_pred, average= a)
    r = recall_score(Y_test, Y_pred, average= a)
    row = pd.Series({'':a,'Precision':p, 'F1 score':f, 'Recall':r})
    df = df.append(row, ignore_index = True )
    print("Precision: ",p)
    print("F1 score: ", f)
    print("Recall: ", r)
  #cm = confusion_matrix(Y_test, model.predict(X_test))
  #fig, ax = plt.subplots(figsize=(14,14))
  #plot_confusion_matrix(model, X_test, Y_test,cmap=plt.cm.Blues,ax=ax)
  #plt.show()
  # Tạo một dataframe dùng để lưu lại kết quả confusion_matrix
  #da = pd.DataFrame(cm)
  # lưu lại kết quả confusion_matrix tập train
  #da.to_csv("/content/Data/NB/Pipeline_{}_test_cm.csv".format(NB))
  ac = accuracy_score(Y_test,Y_pred, normalize=True)
  acf = accuracy_score(Y_test,Y_pred, normalize=False)
  row = pd.Series({'Accuracy':ac, 'Accuracy_false':acf})
  df = df.append(row, ignore_index = True )
  print("Accuracy: ", ac)
  print("Accuracy_false: ", acf)
df.to_csv('/content/Data/NB/NB_Kq_Pipeline.csv', index = False)

#Random Forest Classification

In [ ]:
import pickle
import pandas
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, accuracy_score

In [ ]:
X_train = pickle.load(open('/content/Data/X_train.pkl', 'rb'))
X_test = pickle.load(open('/content/Data/X_test.pkl', 'rb'))
Y_test = pickle.load(open('/content/Data/Y_test.pkl', 'rb'))
Y_train = pickle.load(open('/content/Data/Y_train.pkl', 'rb'))
X_train_trans = pickle.load(open('/content/Data/X_train_tf.pkl', 'rb'))
X_test_trans = pickle.load(open('/content/Data/X_test_tf.pkl', 'rb'))

In [ ]:
a = ["gini", "entropy"]
av = ['micro', 'weighted', 'macro']
for i in range(1,100):
  for j in a:
    for k in av:
      model = RandomForestClassifier(n_estimators=i, criterion = j)
      model.fit(X_train_trans, Y_train)
      score = f1_score(Y_test, model.predict(X_test_trans), average = "micro")
      print(i, j , score)
#title = "CM"
#fig, ax = plt.subplots(figsize=(14,14))
#disp = plot_confusion_matrix(model, X_test, Y_test,cmap=plt.cm.Blues,ax=ax)
#disp.ax_.set_title(title)
#print(title)
#print(disp.confusion_matrix)

In [ ]:
a = ["gini", "entropy"]
for i in range(1,100):
  for j in a:
    model = RandomForestClassifier(n_estimators=i, criterion = j)
    model.fit(X_train_trans, Y_train)
    score = precision_score(Y_test, model.predict(X_test_trans), average = "micro")
    print(i, j , score)
#title = "CM"
#fig, ax = plt.subplots(figsize=(14,14))
#disp = plot_confusion_matrix(model, X_test, Y_test,cmap=plt.cm.Blues,ax=ax)
#disp.ax_.set_title(title)
#print(title)
#print(disp.confusion_matrix)

In [ ]:
a = ["gini", "entropy"]
for i in range(1,100):
  for j in a:
    model = RandomForestClassifier(n_estimators=i, criterion = j)
    model.fit(X_train_trans, Y_train)
    score = recall_score(Y_test, model.predict(X_test_trans), average = "micro")
    print(i, j , score)
#title = "CM"
#fig, ax = plt.subplots(figsize=(14,14))
#disp = plot_confusion_matrix(model, X_test, Y_test,cmap=plt.cm.Blues,ax=ax)
#disp.ax_.set_title(title)
#print(title)
#print(disp.confusion_matrix)

In [ ]:
a = ["gini", "entropy"]
for i in range(1,100):
  for j in a:
    model = RandomForestClassifier(n_estimators=i, criterion = j)
    model.fit(X_train_trans, Y_train)
    score = accuracy_score(Y_test, model.predict(X_test_trans), normalize="True")
    print(i, j , score)
#title = "CM"
#fig, ax = plt.subplots(figsize=(14,14))
#disp = plot_confusion_matrix(model, X_test, Y_test,cmap=plt.cm.Blues,ax=ax)
#disp.ax_.set_title(title)
#print(title)
#print(disp.confusion_matrix)

#LSTM

In [ ]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

In [ ]:
# Đọc dữ liệu
X_train = pickle.load(open('Data/X_train.pkl', 'rb'))
X_test = pickle.load(open('Data/X_test.pkl', 'rb'))
Y_test = pickle.load(open('Data/Y_test.pkl', 'rb'))
Y_train = pickle.load(open('Data/Y_train.pkl', 'rb'))

In [ ]:
tfidf_vect = TfidfVectorizer(analyzer='word', max_features=30000)
tfidf_vect.fit(X_train)
X_data_tfidf =  tfidf_vect.transform(X_train)
X_test_tfidf =  tfidf_vect.transform(X_test)

In [ ]:
svd = TruncatedSVD(n_components=1000, random_state=42)
svd.fit(X_data_tfidf)

X_data_tfidf_svd = svd.transform(X_data_tfidf)
X_test_tfidf_svd = svd.transform(X_test_tfidf)

In [ ]:
encoder = LabelEncoder()
y_data_n = encoder.fit_transform(Y_train)
y_test_n = encoder.fit_transform(Y_test)

encoder.classes_

In [ ]:
def train_model(classifier, X_data, y_data, X_test, y_test, is_neuralnet=False, n_epochs=3):       
    X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.1, random_state=42)
    
    if is_neuralnet:
        classifier.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=n_epochs, batch_size=512)
        
        val_predictions = classifier.predict(X_val)
        test_predictions = classifier.predict(X_test)
        val_predictions = val_predictions.argmax(axis=-1)
        test_predictions = test_predictions.argmax(axis=-1)

    print("Validation accuracy: ", accuracy_score(val_predictions, y_val))
    print("Test accuracy: ", accuracy_score(test_predictions, y_test))

In [ ]:
def create_lstm_model():
    input_layer = Input(shape=(1000,))
    layer = Reshape((1, 1000))(input_layer)
    layer = LSTM(512, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    output_layer = Dense(27, activation='softmax')(layer)
    
    classifier = Model(input_layer, output_layer)
    classifier.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return classifier

In [ ]:
classifier = create_lstm_model()
train_model(classifier=classifier, X_data=X_data_tfidf_svd, y_data=y_data_n, X_test=X_test_tfidf_svd, y_test=y_test_n, is_neuralnet=True, n_epochs= 10)